In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

In [6]:
import pandas as pd
import json

In [3]:
raw_data = pd.read_csv('amazon_reviews_multilingual_US_v1_00.tsv', delimiter='\t', on_bad_lines='warn')
# print(raw_data)

Skipping line 3231472: expected 15 fields, saw 22

Skipping line 3509762: expected 15 fields, saw 22

Skipping line 4018793: expected 15 fields, saw 22

Skipping line 4280173: expected 15 fields, saw 22
Skipping line 4290596: expected 15 fields, saw 22

Skipping line 4331421: expected 15 fields, saw 22
Skipping line 4340267: expected 15 fields, saw 22
Skipping line 4341665: expected 15 fields, saw 22
Skipping line 4386155: expected 15 fields, saw 22
Skipping line 4388098: expected 15 fields, saw 22

Skipping line 4408027: expected 15 fields, saw 22
Skipping line 4442615: expected 15 fields, saw 22

Skipping line 4519623: expected 15 fields, saw 22

Skipping line 4525797: expected 15 fields, saw 22
Skipping line 4543519: expected 15 fields, saw 22

Skipping line 4587726: expected 15 fields, saw 22
Skipping line 4589301: expected 15 fields, saw 22
Skipping line 4634393: expected 15 fields, saw 22

Skipping line 4666168: expected 15 fields, saw 22

Skipping line 4725035: expected 15 field

In [33]:
column_headers = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
raw_data_changed_column = raw_data
raw_data.columns = column_headers
raw_data_100 = raw_data.head(100)
# print(raw_data_10000)

In [34]:
data_grouped_by_customer_id = dict()
data_grouped_by_product_id = dict()
for index, row in raw_data_100.iterrows():
    # review_dict = dict()

    row_dict = row.to_dict()

    # review_dict['customer_id'] = row['customer_id']
    # review_dict['product_id'] = row['product_id']
    # review_dict['product_title'] = row['product_title']
    # review_dict['product_category'] = row['product_category']
    # review_dict['review_id'] = row['review_id']
    # review_dict['review_body'] = row['review_body']
    # review_dict['star_rating'] = row['star_rating']

    if row['customer_id'] in data_grouped_by_customer_id:
        data_grouped_by_customer_id[row['customer_id']].append(row_dict)
    else:
        data_grouped_by_customer_id[row['customer_id']] = [row_dict]

    if row['product_id'] in data_grouped_by_product_id:
        data_grouped_by_product_id[row['product_id']].append(row_dict)
    else:
        data_grouped_by_product_id[row['product_id']] = [row_dict]

In [25]:
print(len(data_grouped_by_customer_id))
for key, value in data_grouped_by_customer_id.items():
    if len(value) > 20:
        print(key, value)
        print()

8370
52779939 [{'marketplace': 'US', 'customer_id': 52779939, 'review_id': 'R37KVBLT0U933B', 'product_id': 'B000002GY9', 'product_parent': 816617040, 'product_title': 'Small Change', 'product_category': 'Music', 'star_rating': 5.0, 'helpful_votes': 21.0, 'total_votes': 24.0, 'vine': 'N', 'verified_purchase': 'N', 'review_headline': 'sinfully pretty and blue', 'review_body': "OHMIGOD this c.d. is choice.  I am listening to it right now.  There is a sensual beauty in the music but the real brilliance comes out in the lyrics.  &quot;You ask me what I'm doin' here, hold this old lamppost, flippin' this old quarter, tryin' to make up my mind... If it's heads I'll go to Tennessee, if it's tails I'll buy a drink, if it lands on the edge I'll keep talkin'... to... you...&quot;   <BR> Waits' nostalgia and grinning, drunken comraderie come in his devastating growl and the pure poetry of his language.  Gloomy yet headshakingly happy.  &quot;The Piano Has Been Drinking&quot; is one of the best son

In [30]:
print(len(data_grouped_by_product_id))
for key, value in data_grouped_by_product_id.items():
    if len(value) > 20:
        print(key, value[:1])
        print()

2197
0517122707 [{'marketplace': 'US', 'customer_id': 51747709, 'review_id': 'R1P5J3FNBWTFXY', 'product_id': '0517122707', 'product_parent': 161411385, 'product_title': 'A Confederacy of Dunces', 'product_category': 'Books', 'star_rating': 5.0, 'helpful_votes': 0.0, 'total_votes': 0.0, 'vine': 'N', 'verified_purchase': 'N', 'review_headline': "A winner that didn't last. Only a mothers love remained", 'review_body': 'The story behind the book is almost better than the work. But make no mistake, the work will endure.', 'review_date': '1995-10-17'}]

0345350499 [{'marketplace': 'US', 'customer_id': 53095881, 'review_id': 'R2XQTCOBWNCEAC', 'product_id': '0345350499', 'product_parent': 627223385, 'product_title': 'The Mists of Avalon', 'product_category': 'Books', 'star_rating': 5.0, 'helpful_votes': 0.0, 'total_votes': 0.0, 'vine': 'N', 'verified_purchase': 'N', 'review_headline': "One of the best books I've ever read!", 'review_body': "Marion Zimmer Bradley brings King Arthur's court to l

In [31]:
from transformers import pipeline
import numpy as np
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [39]:
for product, reviews in data_grouped_by_product_id.items():
    for review in reviews:
        review['sentiment'] = classifier('This is the greatest thing ever')[0]

print(json.dumps(data_grouped_by_product_id, indent=4))

{
    "1563890119": [
        {
            "marketplace": "US",
            "customer_id": 53096384,
            "review_id": "R63J84G1LOX6R",
            "product_id": "1563890119",
            "product_parent": 763187671,
            "product_title": "The Sandman Vol. 1: Preludes and Nocturnes",
            "product_category": "Books",
            "star_rating": 4.0,
            "helpful_votes": 0.0,
            "total_votes": 1.0,
            "vine": "N",
            "verified_purchase": "N",
            "review_headline": "ignore the review below",
            "review_body": "this is the first 8 issues of the series. it is the starting point of all this... it also contains the sound of her wings. issue #8. which is the first appearance of death. and many peoples favorite issue. its not the best of the collected works. but its the start of them.",
            "review_date": "1995-08-13",
            "sentiment": {
                "label": "POSITIVE",
                "score": 0.9998